In [17]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from glob import glob
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import re
import datetime

# スプレッドシート認証設定
SPREADSHEET_ID = "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, grape_calculator_myfive, assign_area):
    print(f'データ前処理を行います')
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre = df_pre[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calculator_myfive(df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    print(f'以下のモデルが含まれています')
    for i, model in enumerate(model_list):
        print(f'{i}: {model}', end=", ")
    return df_pre


def create_df_from_database(HALL_NAME, DB_PATH):
    # Table name 取得
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
            print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,     -- ホール名を追加
            m.name AS model_name     -- 機種名を追加
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
        JOIN models m ON r.model_id = m.model_id  -- 機種と結合
        WHERE h.name = ?  -- 指定ホールのみ
        AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
        ORDER BY r.date DESC, r.unit_no ASC;
        """

    df = pd.read_sql_query(query, conn, params=(hall_name,))
    conn.close()
    print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
    
    return df


def create_pivot_table_date(
    start_date, end_date, day_targets, pivot_targets, index_targets, colmns_targets
):
    """日付ごとにシートを作成して台ごとの出玉率一覧を作成"""
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    df_filtered = df_filtered[df_filtered["day"] == day_targets]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=colmns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="Total",
        )
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate

In [18]:
HALL_NAME = "EXA FIRST"
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
df = create_df_from_database(HALL_NAME, DB_PATH)
df = df_preprocessing(df, grape_calculator_myfive, assign_area)
df.head(3)

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 61104 x 11
データ前処理を行います
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area
0,EXA FIRST,2025-05-09,ゴーゴージャグラー3,1001,3655,10,12,-865,365.5,304.6,5.96,166.1,2025-05,9,4,2025,1,a: 1001-
1,EXA FIRST,2025-05-09,ゴーゴージャグラー3,1002,2064,9,3,-226,229.3,688.0,6.37,172.0,2025-05,9,4,2025,2,a: 1001-
2,EXA FIRST,2025-05-09,ゴーゴージャグラー3,1003,2066,4,9,-674,516.5,229.6,5.96,158.9,2025-05,9,4,2025,3,a: 1001-


## 出力データ
- 期間指定
    - 機種別の分析
    - 島別の分析
    - 台番号分析
    - 日付分析
    - 月分析
    - 過去n日の差枚比較
    - 全台系分析

## ピボットテーブル作成

In [4]:
def create_pivot_table(df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path, hall_name=HALL_NAME):
    df_filtered = df.copy()
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]

    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=index_targets,
            columns=colmns_targets,
            values=col,
            aggfunc="sum",
            margins=True,
            margins_name="total",
        )
        pivot_results[col] = table
    # pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals+game*3) / (game*3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([
        game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[interleaved_cols]
    merged.to_csv(csv_path)

    return merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate

## MEDAL_RATE

In [5]:
today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(months=20, days=start_date.day - 1)
print(f"対象期間: {start_date} から {end_date}")

csv_path = f"{HALL_NAME}_medal_rate.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
# medal_rate.to_csv(csv_path)

target_rate = 1.05
medal_rate[("MEDAL_RATE", f"count_{target_rate}+")] = (medal_rate.iloc[:, :-1] >= target_rate).sum(axis=1)
countif = (medal_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
medal_rate = pd.concat([medal_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0)

rows, cols = medal_rate.shape
sheet_name = f"MEDAL_RATE"
try:
    worksheet = spreadsheet.worksheet(sheet_name)
    print(f"✅ シート「{sheet_name}」が既に存在します。")
except gspread.exceptions.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
    print(f"🆕 シート「{sheet_name}」を新規作成しました。")

sheet = spreadsheet.worksheet(sheet_name)
sheet.clear()
set_with_dataframe(sheet, medal_rate, include_index=True)
sheet.update_cell(1, 1, today.strftime("UPDATED: %Y-%m-%d"))
print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

medal_rate.tail()


対象期間: 2025-05-10 から 2023-09-01
✅ シート「MEDAL_RATE」が既に存在します。
✅ シート 'MEDAL_RATE' に DataFrame を書き込みました！


MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
t: 1151-    1153      0.989  1.006  1.022  0.955  0.994  1.006  0.995  0.982   
            1154      1.027  0.968  1.008  0.924  1.008  0.980  1.015  1.038   
            1155      1.005  0.961  0.972  0.964  1.017  1.003  0.981  1.003   
total                 1.016  1.004  1.003  0.999  1.014  1.000  1.002  1.007   
count_1.05+           7.000  4.000  1.000  4.000  5.000  3.000  3.000  8.000   

                                ...                                      \
day                   9     10  ...     24     25      26     27     28   
t: 1151-    1153  1.026  1.037  ...  1.010  0.990   1.006  1.001  1.025   
            1154  0.961  0.948  ...  1.024  1.015   1.013  1.018  0.983   
            1155  1.011  1.005  ...  0.970  1.019   1.002  0.968  0.949   
total             1.002  1.005  ...  1.000  1.014   1.018  1.006  1.000   
count_1.05+       4.000  8.000  ...  5.000  8.000  10.000  8.000  5.000   

                                                           
day                  29     30      31  total count_1.05+  
t: 1151-    1153  1.031  1.001   0.995  1.006           0  
            1154  1.000  1.005   1.020  1.005           0  
            1155  1.038  0.997   1.030  0.996           0  
total             1.003  1.002   1.007  1.006           0  
count_1.05+       4.000  4.000  11.000  0.000          40  

[5 rows x 33 columns]

## 島分析

In [19]:
csv_path = f"{HALL_NAME}_area.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
# medal_rate.to_csv(csv_path)
merged.head(3)

,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,...,MEDAL_RATE,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,MEDAL_RATE,RB_RATE,TOTAL_RATE
day,1,1,1,1,1,1,1,2,2,2,...,31,31,31,total,total,total,total,total,total,total
area,,,,,,,,,,,,,,,,,,,,,
a: 1001-,384978,44788,1585,1200,1.039,320.8,138.2,253327,-6885,960,...,1.004,347.4,149.9,7653259,86105,29180,21993,1.004,348.0,149.6
b: 1011-,392273,22110,1499,1286,1.019,305.0,140.9,284390,-4995,1039,...,0.981,328.4,153.9,7869689,72164,29812,23099,1.003,340.7,148.7
c: 1021-,243764,39631,1013,797,1.054,305.9,134.7,169919,-9661,611,...,1.004,386.8,153.7,4842973,40845,18320,13734,1.003,352.6,151.1


## 台分析

In [44]:
csv_path = f"{HALL_NAME}_unit.csv"
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["day"]
merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table(
    df, start_date, end_date, pivot_targets, index_targets, colmns_targets, csv_path)
# medal_rate.to_csv(csv_path)
merged
# for index in merged.index.get_level_values(0).unique():
#     merged_index = merged.xs(index, level="area", drop_level=False).iloc[:, 14:]
#     display(merged_index)

GAME  MEDALS     BB     RB MEDAL_RATE RB_RATE TOTAL_RATE  \
day                     1       1      1      1          1       1          1   
area     unit_no                                                                
a: 1001- 1001       42311    6141    185    119      1.048   355.6      139.2   
         1002       34168    4895    138    121      1.048   282.4      131.9   
         1003       40333    7389    173    127      1.061   317.6      134.4   
         1004       32844    1773    129     96      1.018   342.1      146.0   
         1005       37114    4166    160     98      1.037   378.7      143.9   
...                   ...     ...    ...    ...        ...     ...        ...   
t: 1151- 1152       38445    4823    149    140      1.042   274.6      133.0   
         1153       34025   -1488    113    112      0.985   303.8      151.2   
         1154       35572    2127    132    117      1.020   304.0      142.9   
         1155       35134     831    140     86      1.008   408.5      155.5   
total             5701440  275879  22182  17482      1.016   326.1      143.7   

                     GAME MEDALS     BB  ... MEDAL_RATE RB_RATE TOTAL_RATE  \
day                     2      2      2  ...         31      31         31   
area     unit_no                         ...                                 
a: 1001- 1001       33660  -2394    121  ...      0.963   308.6      155.7   
         1002       18471    428     72  ...      1.027   319.4      139.6   
         1003       25252  -2422     89  ...      0.934   459.4      182.3   
         1004       30988   1759    131  ...      1.007   371.4      155.1   
         1005       21611  -1150     77  ...      1.062   288.6      129.7   
...                   ...    ...    ...  ...        ...     ...        ...   
t: 1151- 1152       19555   -365     78  ...      0.948   318.7      163.9   
         1153       26301   -993     95  ...      0.972   299.4      154.0   
         1154       27847  -1051     99  ...      0.969   278.9      150.7   
         1155       30245  -2481    101  ...      1.022   312.3      144.1   
total             4229154 -22711  16109  ...      0.998   342.3      149.7   

                       GAME MEDALS      BB      RB MEDAL_RATE RB_RATE  \
day                   total  total   total   total      total   total   
area     unit_no                                                        
a: 1001- 1001        862631 -42165    3151    2396      0.984   360.0   
         1002        719586  12484    2745    2090      1.006   344.3   
         1003        713333  23596    2779    2098      1.011   340.0   
         1004        723147  12529    2785    2077      1.006   348.2   
         1005        773573   4247    2909    2232      1.002   346.6   
...                     ...    ...     ...     ...        ...     ...   
t: 1151- 1152        688961  -9626    2526    2017      0.995   341.6   
         1153        688296   4274    2589    1977      1.002   348.2   
         1154        636221  -1291    2356    1892      0.999   336.3   
         1155        788303  -9965    2960    2142      0.996   368.0   
total             116932141  71254  444067  337478      1.000   346.5   

                 TOTAL_RATE  
day                   total  
area     unit_no             
a: 1001- 1001         155.5  
         1002         148.8  
         1003         146.3  
         1004         148.7  
         1005         150.5  
...                     ...  
t: 1151- 1152         151.7  
         1153         150.7  
         1154         149.8  
         1155         154.5  
total                 149.6  

[153 rows x 224 columns]

# 日付、台番号指定
- SHEET_NAME : DAY1，DAY2，… DAY31

In [6]:
today = datetime.date.today()
start_date = datetime.date.today()
end_date = start_date - relativedelta(months=6, days=start_date.day - 1)
print(f"対象期間: {start_date} から {end_date}")

pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area", "unit_no"]
colmns_targets = ["date"]

today = datetime.date.today()
for day_target in range(today.day-1, today.day+1):
# for day_target in range(9, 12):
    merged, game, medals, medal_rate, bb, rb, rb_rate, total_rate = create_pivot_table_date(
        start_date, end_date, day_target, pivot_targets, index_targets, colmns_targets)
    rows, cols = medal_rate.shape
    sheet_name = f"DAY{day_target}"
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(rows+5), cols=str(cols+5))
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    set_with_dataframe(sheet, merged, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d : UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

# merged.head()

対象期間: 2025-05-10 から 2024-11-01
✅ シート「DAY9」が既に存在します。
✅ シート 'DAY9' に DataFrame を書き込みました！
✅ シート「DAY10」が既に存在します。
✅ シート 'DAY10' に DataFrame を書き込みました！


In [9]:
# 各種ピボットテーブル
def create_pivot_tabel(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    df_filtered = df_filtered[
        (df_filtered["date"].dt.date <= start_date)
        & (df_filtered["date"].dt.date >= end_date)
    ]
    pivot_targets = ["game", "medals", "RB_rate", "Total_rate", "BB", "RB"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
            index=["model_name", "area", "unit_no"],
            columns="day",
            values=col,
            aggfunc="sum",
        )
        # 日付列を反転・スライス
        # pivot_results[col] = table
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]

    merged = pd.concat([game, medals, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]

    return merged, game, medals, bb, rb, rb_rate, total_rate


start_date = datetime.date.today()
# start_date = datetime.date(2024, 5, 6)
end_date = start_date - relativedelta(days=30)
print(f"{start_date} to {end_date}")

merged, game, medals, bb, rb, rb_rate, total_rate = create_pivot_tabel(
    df, "マイジャグラーV", start_date, end_date
)
merged = merged[~(merged.iloc[:, 0].isna() | (merged.iloc[:, 0] == 0))]
merged.head()

2025-05-09 to 2025-04-09


GAME  MEDALS    BB    RB RB_RATE TOTAL_RATE  \
day                              30      30    30    30      30         30   
model_name area     unit_no                                                  
マイジャグラーV   c: 1021- 1021     6977.0 -2603.0  14.0  23.0   303.3      188.6   
                    1022     5396.0  -903.0  19.0  12.0   449.7      174.1   
                    1023      927.0  -456.0   2.0   2.0   463.5      231.8   
                    1024     7662.0  1738.0  31.0  31.0   247.2      123.6   
                    1025     3007.0  -468.0  10.0   9.0   334.1      158.3   

                               GAME  MEDALS    BB    RB  ...    BB    RB  \
day                              29      29    29    29  ...    2     2    
model_name area     unit_no                              ...               
マイジャグラーV   c: 1021- 1021     6770.0    85.0  24.0  20.0  ...  18.0  21.0   
                    1022     6454.0 -1241.0  23.0   9.0  ...   4.0   6.0   
                    1023     5843.0   865.0  24.0  19.0  ...  11.0   8.0   
                    1024     1303.0  -862.0   3.0   1.0  ...   7.0   6.0   
                    1025     6984.0  1276.0  29.0  19.0  ...   2.0   0.0   

                            RB_RATE TOTAL_RATE    GAME  MEDALS    BB    RB  \
day                              2          2       1       1     1     1    
model_name area     unit_no                                                  
マイジャグラーV   c: 1021- 1021      323.1      174.0  6803.0 -1229.0  20.0  22.0   
                    1022      372.2      223.3  8704.0  5206.0  49.0  36.0   
                    1023      440.2      185.4  7751.0  1488.0  33.0  24.0   
                    1024      410.2      189.3  6661.0  1544.0  27.0  24.0   
                    1025        0.0      589.0  7689.0  2268.0  35.0  24.0   

                            RB_RATE TOTAL_RATE  
day                              1          1   
model_name area     unit_no                     
マイジャグラーV   c: 1021- 1021      309.2      162.0  
                    1022      241.8      102.4  
                    1023      323.0      136.0  
                    1024      277.5      130.6  
                    1025      320.4      130.3  

[5 rows x 180 columns]

In [10]:
model_list = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
    # "ミスタージャグラー",
    # "ウルトラミラクルジャグラー",
    # "ハッピージャグラーVIII",
    # "ジャグラーガールズ",
]

start_date = datetime.date.today()
end_date = start_date - relativedelta(days=60)
print(f"{start_date} to {end_date}")

merged_models = pd.DataFrame()
for model in list(model_list):
    print(model)
    merged, game, medals, bb, rb, rb_rate, total_rate = create_pivot_tabel(df, model, start_date, end_date)
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        area_merged = area_merged[~area_merged.iloc[:, 0].isna()]
        empty_index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
        empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
        
        # if area_merged.shape[0]:
        #     summary_row = []
        #     for c in area_merged.columns:
        #         if c[0] in ["GAME", "MEDALS", "RB", "BB"]:
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             summary_row.append(area_merged[c].sum())
        #         elif c[0] == "RB_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             rb_rate = game_sum / rb_sum if rb_sum != 0 else None
        #             summary_row.append(round(rb_rate, 1) if rb_rate else None)
        #         elif c[0] == "TOTAL_RATE":
        #             game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        #             rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        #             bb_sum = area_merged.get(("BB", c[1]), pd.Series()).sum()
        #             total_rate = game_sum / (rb_sum + bb_sum) if (rb_sum + bb_sum) != 0 else None
        #             summary_row.append(round(total_rate, 1) if total_rate else None)
            

            # summary_df = pd.DataFrame(summary_row).T
            # summary_df.index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
            # summary_df.columns = area_merged.columns
        merged_models = pd.concat([merged_models, area_merged, empty_row])

merged_models = merged_models[~(merged_models.iloc[:, 0].isna() | (merged_models.iloc[:, 0] == 0))]
merged_models.to_csv(f"{HALL_NAME}_unit.csv")
    
merged_models.head(10)

2025-05-09 to 2025-03-10
マイジャグラーV


GAME MEDALS  BB  RB RB_RATE TOTAL_RATE  GAME  \
day                            31     31  31  31      31         31    30   
model_name area     unit_no                                                 
マイジャグラーV   a: 1001- 1001     7407    865  27  33   224.5      123.4  2920   
                    1002     6947   2003  32  26   267.2      119.8  8390   
                    1003     3610   -503  11  10   361.0      171.9  1553   
                    1004     1052   -438   3   1  1052.0      263.0  3155   
                    1005     9461   3612  44  40   236.5      112.6  6203   
                    1006     1037   -621   2   1  1037.0      345.7  7317   
                    1007     3913   -462  14   9   434.8      170.1  4843   
                    1008     6506    124  24  21   309.8      144.6  2808   
                    1009     6350   -253  24  14   453.6      167.1  1941   
                    1010     2981  -1429   5   9   331.2      212.9  5494   

                            MEDALS  BB  RB  ...  BB  RB RB_RATE TOTAL_RATE  \
day                             30  30  30  ...  2   2       2          2    
model_name area     unit_no                 ...                              
マイジャグラーV   a: 1001- 1001       144  12   6  ...  26  13   413.5      137.8   
                    1002       665  29  32  ...  12  18   193.7      116.2   
                    1003         3   6   4  ...   4   2   612.0      204.0   
                    1004     -1065   8   9  ...   2   2   431.0      215.5   
                    1005      -218  24  15  ...  34  25   270.0      114.4   
                    1006      -712  24  21  ...   5   5   269.2      134.6   
                    1007      -638  18   5  ...   9   4   485.0      149.2   
                    1008      -547   7   9  ...   6   2   948.5      237.1   
                    1009      -315   7   4  ...   3   3   408.0      204.0   
                    1010      -768  17  17  ...  14   6   612.0      183.6   

                             GAME MEDALS  BB  RB RB_RATE TOTAL_RATE  
day                            1      1   1   1       1          1   
model_name area     unit_no                                          
マイジャグラーV   a: 1001- 1001     6879    344  24  24   286.6      143.3  
                    1002     1441   -450   5   3   480.3      180.1  
                    1003     7691   3306  42  18   427.3      128.2  
                    1004     4512    226  19  11   410.2      150.4  
                    1005     1852   -162   9   1  1852.0      185.2  
                    1006     3912   -662  13  10   391.2      170.1  
                    1007     3854    -15  15   7   550.6      175.2  
                    1008     4801   -874  16   8   600.1      200.0  
                    1009     6071    276  23  16   379.4      155.7  
                    1010     5913   1100  26  18   328.5      134.4  

[10 rows x 186 columns]

In [11]:
len(area_merged.columns)

186